In [4]:
import pandas as pd
import random

def load_dataset(file_path):
    """Load the dataset from CSV."""
    df = pd.read_csv(file_path)
    
    # Ensure numerical columns are correctly interpreted
    numeric_cols = ["Calories", "Carbohydrates", "Protein", "Fat"]
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors="coerce")
    
    return df

def filter_data(df, diet_type, cuisine, goal):
    """Filter dataset based on user input and nutritional constraints."""
    goal_constraints = {
        "Muscle Gain": {"Calories": (400, 700), "Carbohydrates": (30, 100), "Protein": (20, 50), "Fat": (10, 35)},
        "Weight Loss": {"Calories": (150, 450), "Carbohydrates": (10, 60), "Protein": (5, 30), "Fat": (3, 20)},
        "Balanced Diet": {"Calories": (250, 600), "Carbohydrates": (20, 70), "Protein": (10, 40), "Fat": (5, 25)}
    }
    
    constraints = goal_constraints[goal]
    filtered_df = df[(df["Diet"] == diet_type) & (df["Cuisine"] == cuisine)]
    
    # Apply constraints
    for col, (min_val, max_val) in constraints.items():
        filtered_df = filtered_df[(filtered_df[col] >= min_val) & (filtered_df[col] <= max_val)]
    
    return filtered_df

def get_alternative_meal(df, diet_type, meal_type):
    """Get an alternative meal from another cuisine but same meal type and diet type."""
    alternative_meals = df[(df["Diet"] == diet_type) & (df["Meal Type"].str.contains(meal_type, case=False, na=False))]
    
    if not alternative_meals.empty:
        return alternative_meals.sample(n=1).iloc[0]["Food Name"]
    
    return "No Alternative Found"

def create_weekly_plan(filtered_df, df, diet_type, cuisine, goal):
    """Generate a 7-day meal plan covering all meal types."""
    weekly_plan = {}
    
    for day in ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]:
        daily_plan = {}

        for meal_type in ["Breakfast", "Lunch", "Snack", "Dinner"]:
            meal_options = filtered_df[filtered_df["Meal Type"].str.contains(meal_type, case=False, na=False)]
            
            if meal_options.empty:
                # Fetch a meal from another cuisine but with same meal type and diet type
                daily_plan[meal_type] = get_alternative_meal(df, diet_type, meal_type)
            else:
                meal = meal_options.sample(n=1).iloc[0]
                daily_plan[meal_type] = meal["Food Name"]

        weekly_plan[day] = daily_plan

    return weekly_plan

def save_meal_plan(weekly_plan, diet_type, cuisine, goal):
    """Save the meal plan to a CSV file."""
    output_file = f"{diet_type}_{cuisine}_{goal}_weekly_diet_plan.csv"
    
    df = pd.DataFrame([(day, meal_type, meal) for day, meals in weekly_plan.items() for meal_type, meal in meals.items()], 
                      columns=["Day", "Meal Type", "Meal"])
    df.to_csv(output_file, index=False)
    return output_file

# Example usage
file_path = "/Users/vigneshalle/Downloads/indian_food_dataset3.csv"
dataset = load_dataset(file_path)

diet_type = input("Enter Diet Type (Vegetarian/Non-Vegetarian): ")
cuisine = input("Enter Cuisine (North Indian/South Indian/East Indian/West Indian): ")
goal = input("Enter your goal (Muscle Gain/Weight Loss/Balanced Diet): ")

filtered_data = filter_data(dataset, diet_type, cuisine, goal)
weekly_plan = create_weekly_plan(filtered_data, dataset, diet_type, cuisine, goal)
csv_file = save_meal_plan(weekly_plan, diet_type, cuisine, goal)
print(f"Your weekly meal plan has been successfully saved as {csv_file}.")


Enter Diet Type (Vegetarian/Non-Vegetarian):  Vegetarian
Enter Cuisine (North Indian/South Indian/East Indian/West Indian):  North Indian
Enter your goal (Muscle Gain/Weight Loss/Balanced Diet):  Muscle Gain


Your weekly meal plan has been successfully saved as Vegetarian_North Indian_Muscle Gain_weekly_diet_plan.csv.


In [8]:
import pandas as pd
import random
import gradio as gr

def load_dataset(file_path):
    """Load the dataset from CSV."""
    df = pd.read_csv(file_path)
    
    # Ensure numerical columns are correctly interpreted
    numeric_cols = ["Calories", "Carbohydrates", "Protein", "Fat"]
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors="coerce")
    
    return df

def filter_data(df, diet_type, goal):
    """Filter dataset based on user input and nutritional constraints."""
    goal_constraints = {
        "Muscle Gain": {"Calories": (400, 700), "Carbohydrates": (30, 100), "Protein": (20, 50), "Fat": (10, 35)},
        "Weight Loss": {"Calories": (150, 450), "Carbohydrates": (10, 60), "Protein": (5, 30), "Fat": (3, 20)},
        "Balanced Diet": {"Calories": (250, 600), "Carbohydrates": (20, 70), "Protein": (10, 40), "Fat": (5, 25)}
    }
    
    constraints = goal_constraints[goal]
    filtered_df = df[df["Diet"] == diet_type]
    
    # Apply constraints
    for col, (min_val, max_val) in constraints.items():
        filtered_df = filtered_df[(filtered_df[col] >= min_val) & (filtered_df[col] <= max_val)]
    
    return filtered_df

def get_alternative_meal(df, diet_type, meal_type):
    """Get an alternative meal from the dataset with the same meal type and diet type."""
    alternative_meals = df[(df["Diet"] == diet_type) & (df["Meal Type"].str.contains(meal_type, case=False, na=False))]
    
    if not alternative_meals.empty:
        return alternative_meals.sample(n=1).iloc[0]["Food Name"]
    
    return "No Alternative Found"

def create_weekly_plan(filtered_df, df, diet_type, goal):
    """Generate a 7-day meal plan covering all meal types."""
    weekly_plan = []
    
    for day in ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]:
        for meal_type in ["Breakfast", "Lunch", "Snack", "Dinner"]:
            meal_options = filtered_df[filtered_df["Meal Type"].str.contains(meal_type, case=False, na=False)]
            
            if meal_options.empty:
                meal = get_alternative_meal(df, diet_type, meal_type)
            else:
                meal = meal_options.sample(n=1).iloc[0]["Food Name"]
            
            weekly_plan.append([day, meal_type, meal])
    
    return pd.DataFrame(weekly_plan, columns=["Day", "Meal Type", "Meal"])

def save_meal_plan(df, diet_type, goal):
    """Save the meal plan to a CSV file."""
    output_file = f"{diet_type}_{goal}_weekly_diet_plan.csv"
    df.to_csv(output_file, index=False)
    return output_file

def generate_meal_plan(diet_type, goal):
    dataset = load_dataset("indian_food_dataset3.csv")
    filtered_data = filter_data(dataset, diet_type, goal)
    weekly_plan_df = create_weekly_plan(filtered_data, dataset, diet_type, goal)
    csv_file = save_meal_plan(weekly_plan_df, diet_type, goal)
    
    return weekly_plan_df, gr.File(value=csv_file, label="Download Your Meal Plan")

demo = gr.Interface(
    fn=generate_meal_plan,
    inputs=[
        gr.Radio(["Vegetarian", "Non-Vegetarian"], label="Diet Type"),
        gr.Radio(["Muscle Gain", "Weight Loss", "Balanced Diet"], label="Goal")
    ],
    outputs=[gr.Dataframe(headers=["Day", "Meal Type", "Meal"]), gr.File()],
    title="🍴Personalized Diet Plan Recommender 🍽️ ",
    description="Select your preferences and get a customized 7-day meal plan."
)

demo.launch()


* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
